<h1 style="font-size: 22px; line-height: 100%; text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142); border-radius: 10px;">
External Sub-network Search with BootstrapNAS and DyNAS-T
</h1>

This notebook demonstrates how to use an external search solution to find high-performing subnetworks in NNCF's [BootstrapNAS](https://github.com/openvinotoolkit/nncf/tree/develop/examples/experimental/torch/classification). We will use an existing super-network generated from ResNet-50.  

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Imports and Settings
</h3>

In [1]:
import sys
import os
sys.path.append('..')
sys.path.append('path-to-nncf')

from imports_bnas import * # Import NNCF, PyTorch and other required packages.
import os
from examples.common.sample_config import SampleConfig
from nncf.common.initialization.batchnorm_adaptation import BatchnormAdaptationAlgorithm
from nncf.experimental.torch.nas.bootstrapNAS.search.supernet import TrainedSuperNet
from nncf.experimental.torch.nas.bootstrapNAS.elasticity.multi_elasticity_handler import SubnetConfig
from nncf.experimental.torch.nas.bootstrapNAS.elasticity.elasticity_dim import ElasticityDim

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

MODEL_DIR = Path("../../../models/supernets/cifar10/resnet50")
OUTPUT_DIR = Path("output")
DATA_DIR = Path("data")
BASE_MODEL_NAME = "resnet-50"

fp32_pth_path, model_onnx_path, supernet_onnx_path, subnet_onnx_path = create_folders_demo(BASE_MODEL_NAME + '_external_search')


Imported PyTorch and NNCF
INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch


<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Loading an existing super-network
</h3>

In [ ]:
model = resnet50_cifar10()
config = SampleConfig.from_json(os.path.join(MODEL_DIR, 'config.json'))
config.device = device
print(MODEL_DIR)
supernet = TrainedSuperNet.from_checkpoint(model, config, os.path.join(MODEL_DIR, 'elasticity.pth'), os.path.join(MODEL_DIR, 'supernet_weights.pth'))

../../../models/supernets/cifar10/resnet50


<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Explore the Search Space
</h3>



In [ ]:
supernet.get_search_space()

{'width': {0: [256, 200, 152, 128],
  1: [512, 408, 304, 256],
  2: [1024, 816, 608, 512],
  3: [2048, 1632, 1224, 1024],
  4: [64, 48, 32],
  5: [64, 48, 32],
  6: [64, 48, 32],
  7: [64, 48, 32],
  8: [64, 48, 32],
  9: [64, 48, 32],
  10: [64, 48, 32],
  11: [128, 96, 72, 64],
  12: [128, 96, 72, 64],
  13: [128, 96, 72, 64],
  14: [128, 96, 72, 64],
  15: [128, 96, 72, 64],
  16: [128, 96, 72, 64],
  17: [128, 96, 72, 64],
  18: [128, 96, 72, 64],
  19: [256, 200, 152, 128],
  20: [256, 200, 152, 128],
  21: [256, 200, 152, 128],
  22: [256, 200, 152, 128],
  23: [256, 200, 152, 128],
  24: [256, 200, 152, 128],
  25: [256, 200, 152, 128],
  26: [256, 200, 152, 128],
  27: [256, 200, 152, 128],
  28: [256, 200, 152, 128],
  29: [256, 200, 152, 128],
  30: [256, 200, 152, 128],
  31: [512, 408, 304, 256],
  32: [512, 408, 304, 256],
  33: [512, 408, 304, 256],
  34: [512, 408, 304, 256],
  35: [512, 408, 304, 256],
  36: [512, 408, 304, 256]},
 'depth': [[0],
  [2],
  [4],
  [5],
  

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Data Loaders for validation and batchnorm adaptation
</h3>

Note: Number of samples used for Batch Norm Adaptation might greatly influence the top1 evaluation for each subnetwork.
`6000` is a good value for ResNet-50 CIFAR10, but it is advised to perform an ablation study for each new architecture to identify the best number of samples.

In [ ]:
DATASET_DIR = DATA_DIR / "cifar10"

batch_size_val = 1000
batch_size = 64
train_loader, val_loader = create_cifar10_dataloader(DATASET_DIR, batch_size, batch_size_val, device)

bn_adapt_args = BNAdaptationInitArgs(data_loader=wrap_dataloader_for_init(train_loader), device=config.device)

bn_adapt_algo_kwargs = {
    'data_loader': train_loader,
    'num_bn_adaptation_samples': 6000,
    'device': 'cuda',
}
bn_adaptation = BatchnormAdaptationAlgorithm(**bn_adapt_algo_kwargs)

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Sample sub-networks to implement your own algorithm
</h3>


In [ ]:
# Implement your function to validate 

def acc_top1(model):
    from bootstrapnas_utils import validate

    bn_adaptation.run(model)

    model.eval()
    model.to(device)
    model_top1_acc, _, _ = validate(model, val_loader)
    return model_top1_acc.detach().cpu().numpy()

supernet.activate_maximal_subnet()
supernet.eval_active_subnet(acc_top1)

Test: [ 0/10]	Time 0.671 (0.671)	Loss 0.314 (0.314)	Acc@1 93.30 (93.30)	Acc@5 99.00 (99.00)
 * Acc@1 93.660 Acc@5 98.990


array(93.659996, dtype=float32)

In [ ]:
subnet_config = SubnetConfig()
subnet_config[ElasticityDim.DEPTH] = [1]

supernet.eval_subnet(subnet_config, acc_top1)

Test: [ 0/10]	Time 0.605 (0.605)	Loss 0.313 (0.313)	Acc@1 93.40 (93.40)	Acc@5 98.90 (98.90)
 * Acc@1 93.670 Acc@5 99.030


array(93.67, dtype=float32)

In [ ]:
supernet.activate_config(supernet.get_search_space())
supernet.get_macs_for_active_config()

325.799936

In [ ]:
supernet.activate_config(supernet.get_search_space())
supernet.get_macs_for_active_config()

325.799936

In [ ]:
subnet_config[ElasticityDim.DEPTH] = [0]
subnet_config[ElasticityDim.WIDTH] = {2: 512}

supernet.activate_config(subnet_config)
supernet.get_macs_for_active_config()

272.32256

In [ ]:
supernet.get_active_config()

OrderedDict([(<ElasticityDim.WIDTH: 'width'>,
              {0: 256,
               1: 512,
               2: 512,
               3: 2048,
               4: 64,
               5: 64,
               6: 64,
               7: 64,
               8: 64,
               9: 64,
               10: 64,
               11: 128,
               12: 128,
               13: 128,
               14: 128,
               15: 128,
               16: 128,
               17: 128,
               18: 128,
               19: 256,
               20: 256,
               21: 256,
               22: 256,
               23: 256,
               24: 256,
               25: 256,
               26: 256,
               27: 256,
               28: 256,
               29: 256,
               30: 256,
               31: 512,
               32: 512,
               33: 512,
               34: 512,
               35: 512,
               36: 512}),
             (<ElasticityDim.DEPTH: 'depth'>, [0])])

In [ ]:
supernet.eval_subnet(subnet_config, acc_top1)

Test: [ 0/10]	Time 0.572 (0.572)	Loss 0.329 (0.329)	Acc@1 92.70 (92.70)	Acc@5 98.80 (98.80)
 * Acc@1 93.740 Acc@5 98.950


array(93.74, dtype=float32)

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Search using DyNAS-T
</h3>

![DyNAS-T Logo](https://github.com/IntelLabs/DyNAS-T/blob/main/docs/images/dynast_logo.png?raw=true)

DyNAS-T (Dynamic Neural Architecture Search Toolkit) is a super-network neural architecture search NAS optimization package designed for efficiently discovering optimal deep neural network (DNN) architectures for a variety of performance objectives such as accuracy, latency, multiply-and-accumulates, and model size.

To install DyNAS-T you can either install it from PyPI or from source:

1. PyPI: In your active Python environment install DyNAS-T with:
    ```bash
    pip install dynast
    ```
1. Source: Clone DyNAS-T repository with:
    ```bash
    git clone https://github.com/IntelLabs/DyNAS-T.git
    ``` 
    and from the cloned repository run
    ```bash
    pip install -e .
    ```

DyNAS-T is compatible with supernetworks created with the NNCF BootstrapNAS. The search on NNCF BootstrapNAS supernetworks can be performed in just few steps, as shown below.

In [ ]:
from dynast.dynast_manager import DyNAS

dynast_config = {
    'results_path': 'bootstrapnas_resnet50_cifar10_linas_test_nb_1.csv',
    'num_evals': 250,
    'population': 50,
    'seed': 42,
    'supernet': 'bootstrapnas_image_classification',  # this parameter works in a tangent with `bootstrapnas_supernetwork` and should be set to `bootstrapnas_image_classification` for all CNN BootstrapNAS supernetworks;
    'batch_size': config.batch_size,
    'bootstrapnas_supernetwork': supernet,  # pass the instance of nncf.experimental.torch.nas.bootstrapNAS.search.supernet.TrainedSuperNet to the search algorithm;
    'device': config.device,
    'verbose': False,
    'optimization_metrics': ['accuracy_top1', 'macs'],  # define optimization metrics that will guide the search progression;
    'measurements': ['accuracy_top1', 'macs'],  # define measurements that will be collected during the search; This is a superset of optimization metrics;
    'metric_eval_fns': {
        'accuracy_top1': acc_top1,  # register custom evaluation function;
    }
}

dynas = DyNAS(**dynast_config)

[07-10 13:25:42 MainProcess #37389] INFO  text_to_speech.py:34 - Please install tensorboardX: pip install tensorboardX
[07-10 13:25:43 MainProcess #37389] INFO  dynast_manager.py:50 - ========================================
[07-10 13:25:43 MainProcess #37389] INFO  dynast_manager.py:51 - Starting Dynamic NAS Toolkit (DyNAS-T)
[07-10 13:25:43 MainProcess #37389] INFO  dynast_manager.py:52 - ========================================
[07-10 13:25:43 MainProcess #37389] INFO  dynast_manager.py:76 - ----------------------------------------
[07-10 13:25:43 MainProcess #37389] INFO  dynast_manager.py:77 - DyNAS Parameter Inputs:
[07-10 13:25:43 MainProcess #37389] INFO  dynast_manager.py:79 - results_path: bootstrapnas_resnet50_cifar10_linas_test_nb_1.csv
[07-10 13:25:43 MainProcess #37389] INFO  dynast_manager.py:79 - num_evals: 250
[07-10 13:25:43 MainProcess #37389] INFO  dynast_manager.py:79 - population: 50
[07-10 13:25:43 MainProcess #37389] INFO  dynast_manager.py:79 - seed: 42
[07-10 

In [ ]:
# `results` will contain a list of <population> best configurations found during the search;
results = dynas.search()

[07-10 13:25:43 MainProcess #37389] INFO  evaluation_interface.py:49 - (Re)Formatted results file: bootstrapnas_resnet50_cifar10_linas_test_nb_1.csv
[07-10 13:25:43 MainProcess #37389] INFO  evaluation_interface.py:50 - csv file header: ['Sub-network', 'Date', 'Model Parameters', 'Latency (ms)', 'MACs', 'Top-1 Acc (%)']
[07-10 13:25:43 MainProcess #37389] INFO  search_tactic.py:363 - Starting LINAS loop 1 of 5.
[07-10 13:25:43 MainProcess #37389] INFO  search_tactic.py:367 - Evaluating subnetwork 1/50 in loop 1 of 5


Test: [ 0/10]	Time 0.550 (0.550)	Loss 0.340 (0.340)	Acc@1 92.70 (92.70)	Acc@5 98.90 (98.90)
 * Acc@1 93.070 Acc@5 99.210


[07-10 13:25:53 MainProcess #37389] INFO  search_tactic.py:367 - Evaluating subnetwork 2/50 in loop 1 of 5


Test: [ 0/10]	Time 0.590 (0.590)	Loss 0.334 (0.334)	Acc@1 93.10 (93.10)	Acc@5 99.10 (99.10)
 * Acc@1 93.690 Acc@5 99.330


[07-10 13:26:04 MainProcess #37389] INFO  search_tactic.py:367 - Evaluating subnetwork 3/50 in loop 1 of 5


...

[07-10 14:08:07 MainProcess #37389] INFO  search_tactic.py:347 - Updated self.predictor_dict[macs].
[07-10 14:08:07 MainProcess #37389] INFO  search_tactic.py:345 - Training accuracy_top1 predictor.
[07-10 14:08:07 MainProcess #37389] INFO  predictor_manager.py:38 - Building dynamic predictor for accuracy_top1
[07-10 14:08:08 MainProcess #37389] INFO  encoding.py:246 - Training set length=250


<class 'numpy.ndarray'> [93.07     93.689995 92.93     93.079994 93.369995 93.28     93.229996
 93.439995 93.43     93.63     93.659996 93.369995 93.049995 93.09
 93.03     93.09     93.159996 93.13     93.299995 93.17     93.53
 92.869995 93.119995 92.829994 93.32     93.17     92.85     93.409996
 93.81     93.369995 93.42     93.34     93.479996 92.829994 92.92
 93.479996 92.95     92.909996 93.619995 93.34     92.97     93.079994
 93.5      93.21     93.39     93.06     93.159996 93.159996 93.2
 93.06     93.159996 93.43     93.64     93.6      93.509995 93.5
 93.11     93.27     93.619995 93.71     93.52     93.649994 93.439995
 93.09     93.159996 93.57     93.399994 93.07     93.64     93.57
 93.619995 93.479996 93.47     93.46     93.259995 93.18     93.6
 93.409996 93.649994 93.61     93.32     93.28     93.45     93.24
 93.509995 93.56     93.659996 93.35     93.59     93.57     93.71
 93.06     93.54     93.39     93.579994 93.52     93.299995 93.43
 93.619995 93.53     93.2

[07-10 14:08:09 MainProcess #37389] INFO  search_tactic.py:347 - Updated self.predictor_dict[accuracy_top1].
[07-10 14:08:09 MainProcess #37389] INFO  evolutionary.py:100 - Configuring nsga2 algorithm with 2 objectives.
[07-10 14:08:09 MainProcess #37389] INFO  evolutionary.py:334 - Running Search ...


....................................................................................................

[07-10 14:08:13 MainProcess #37389] INFO  evolutionary.py:350 - Success! Search Took 4.565 seconds.
[07-10 14:08:13 MainProcess #37389] INFO  search_tactic.py:526 - Validated model architectures in file: bootstrapnas_resnet50_cifar10_linas_test_nb_1.csv


<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Visualization of DyNAS-T search on the BootstrapNAS supernetwork
</h3>

<center><img src="dynast_bootstrapnas_resnet50_cifar10_example.png" alt="DyNAS-T Search Results" width="1000"/></center>